In [2]:
import pulp
filename = 'data/taxi1000.txtT60D70P100'
adj_list = []
cap_list = []
left = []
right = []
links = []
adj_list = []
with open(filename, 'rt') as f:
    idx = 0
    for line in f:
        d = [int(i) for i in line.strip().split()]
        type_d = d[0]
        cap_d = d[1]
        cap_list.append(cap_d)
        if type_d == 0:
            left.append(idx)
        else:
            right.append(idx)
        if type_d == 0:
            for j in d[2:]:
                links.append(str(idx)+' '+str(j))
        idx += 1
        adj_list.append(d[2:])

In [3]:
x = pulp.LpVariable.dict('links', links, lowBound=0, upBound=1, cat=pulp.LpInteger)

In [4]:
# objective: maximize the matchings
max_matching = pulp.LpProblem('max matching', pulp.LpMaximize)
max_matching += pulp.lpSum([x[l] for l in links])

In [5]:
# constraints:
for l in left:
    l_links = []
    cap_dict = {}
    if len(adj_list[l]) > 0:
        # print(l, adj_list[l])
        for r in adj_list[l]:
            link = str(l) + ' ' + str(r)
            l_links.append(link)
            cap_dict[link] = cap_list[r]
        max_matching += pulp.lpSum([cap_dict[link]*x[link] for link in l_links]) <= cap_list[l]

for r in right:
    r_links = []
    if len(adj_list[r]) > 0:
        for l in adj_list[r]:
            link = str(l) + ' ' + str(r)
            r_links.append(link)
        max_matching += pulp.lpSum([x[link] for link in r_links]) <= 1

In [6]:
max_matching.solve()

1

In [7]:
print(pulp.LpStatus[max_matching.status])

Optimal


In [8]:
opt = 0
for link in links:
    opt += x[link].value()
print(opt)

428.0
